# **Chapter 8: Avoiding Hallucinations**

---
**Lesson:**

Some bad news: Claude sometimes "hallucinates" and makes claims that are untrue or unjustified. The good news: there are techniques you can use to minimize hallucinations.				
Below, we'll go over a few of these techniques, namely:

- Giving Claude the option to say it doesn't know the answer to a question 
- Asking Claude to find the evidence before answering 
---

First we will setup our dependencies

In [3]:
%%capture
#Cell [1]

#langchain
%pip install langchain

#Install libraries 
%pip install --no-build-isolation --force-reinstall \
    "boto3>=1.28.57" \
    "awscli>=1.29.57" \
    "botocore>=1.31.57"

#Import libraries
import json
import os
import sys
import boto3
import botocore

#Set up Bedrock runtime client
bedrock_runtime = boto3.client('bedrock-runtime')
boto3_bedrock = boto3.client('bedrock')

#Set up ModelID (Claude 3 Sonnet in this case)
modelId = "anthropic.claude-3-sonnet-20240229-v1:0"

In [4]:
#Cell [2]

#Setup Model invocation function 
#Code modified from documentation: https://docs.aws.amazon.com/bedrock/latest/userguide/bedrock-runtime_example_bedrock-runtime_Claude3_Text_section.html
def get_response(prompt_data):
    response = bedrock_runtime.invoke_model(
                    modelId=modelId,
                    body=json.dumps(
                        {
                            "anthropic_version": "bedrock-2023-05-31",
                            "max_tokens": 1024,
                            "messages": [
                                {
                                    "role": "user",
                                    "content": [{"type": "text", "text": prompt_data}],
                                }
                            ],
                        }
                    ),
                )
    result = json.loads(response.get("body").read())
    response = result.get("content", [])[0]['text']
    return response

# **Examples:**

**Example 8.1 - Hungry Hippos**

Here is a question about general factual knowledge to which Claude hallucinates an answer because it's trying to be as helpful as possible.

In [5]:
response = get_response("""

Human: Who is the heaviest hippo ever recorded?  

Assistant:""")

print(response)

Unfortunately, I don't have enough information to definitively state who was the heaviest hippo ever recorded. Hippo weights can vary quite a bit based on factors like age, gender, and environmental conditions.

However, here are some details about remarkably large hippos that have been documented:

- In 2012, a male hippo named Humphrey who lived at the Mogen David Winery in Israel was claimed to weigh around 7,000 lbs (3,175 kg), which would make him potentially one of the heaviest ever.

- In 1980, a hippo bull shot in Malawi, Africa reportedly weighed 6,100 lbs (2,766 kg).

- The San Diego Zoo had a male hippo named Graham who reached an estimated 6,000 lbs (2,722 kg) before his death in the 1980s.

- Male hippos in the wild typically weigh between 3,000-6,000 lbs (1,360-2,720 kg) at maximum weight. Females are smaller, ranging from 1,500-3,000 lbs (680-1,360 kg).

So while exact records are difficult to verify conclusively, several hippos over the years have hit extremely high wei

**Example 8.2 - Hungry Hippos 2**

A solution we can try here is to "give Claude an out" — tell Claude that it's OK for it to decline to answer, or to only answer if it actually knows the answer with certainty.

In [6]:
response = get_response("""

Human: Who is the heaviest hippo ever recorded? Only answer if you know the answer with certainty  

Assistant:""")

print(response)

I'm afraid I don't have definitive information on the heaviest hippopotamus ever recorded with full certainty. Hippo weights can vary significantly, and record-keeping on individual animal weights is not always comprehensive or reliable. Unless I can find a clearly cited and trustworthy source stating the record holder, I cannot provide that specific fact with complete confidence.


# **Exercise**

The following exercises will need you to manipulate the prompt to get the desired output

**Exercise 8.1 - NBA Hallucination**

Adapt the prompt in the cell below to fix Claude's hallucination issue by giving Claude an out. (Kawhi Leonard has only won two NBA championships.) 

In [7]:
response = get_response("""

Human: In what year did Kawhi Leonard win his third NBA championship?

Assistant:""")

print(response)

Kawhi Leonard has won two NBA championships so far in his career, not three.

He won his first championship with the San Antonio Spurs in 2014, when they defeated the Miami Heat 4-1 in the NBA Finals.

He then won his second championship with the Toronto Raptors in 2019, when they defeated the Golden State Warriors 4-2 in the Finals. Leonard was named Finals MVP that year.

Leonard has not yet won a third NBA title as of 2023. He is currently playing for the Los Angeles Clippers.


# Chapter 8 - END.